In [1]:
import numpy as np
import cv2
import os
from glob import glob
from tqdm import tqdm
from stereo4d_code.utils import CameraAZ, Track3d
from stereo4d_code.track_optimization import save_3d_track_vis  # Ensure these functions are imported

# --- Step 1: Load annotation data ---
anno_path = "data/pointodyssey/sample/r4_new_f/anno.npz"
data = np.load(anno_path)
# Original shapes:
#   trajs_3d: (849, 11721, 3)   --> frames x points x 3
#   valids:  (849, 11721)       --> frames x points
#   intrinsics: (849, 3, 3)
#   extrinsics: (849, 4, 4)

# Transpose trajectories and visibility arrays so that the shape becomes (points, frames, ...)
trajs_3d = data['trajs_3d'].transpose(1, 0, 2)   # now (11721, 849, 3)
valids = data['valids'].transpose(1, 0)           # now (11721, 849)
# (You can similarly transpose 'visibs' if you prefer to use that.)

# --- Step 2: Load RGB images ---
rgb_files = sorted(glob("data/pointodyssey/sample/r4_new_f/rgbs/*.jpg"))
rgbs = np.array([cv2.imread(f)[:, :, ::-1] for f in tqdm(rgb_files, desc="Loading RGBs")],
                dtype=np.uint8)  # Convert BGR to RGB

# --- Step 3: Load Depth Maps ---
depth_files = sorted(glob("data/pointodyssey/sample/r4_new_f/depths/*.png"))
depths = np.array([cv2.imread(f, cv2.IMREAD_UNCHANGED) for f in tqdm(depth_files, desc="Loading Depths")],
                  dtype=np.float32)
# If your depth images are stored in millimeters, convert them to meters (or adjust as needed):
depths = depths / 1000.0

# Get image dimensions (assuming all images share the same dimensions)
imh, imw, _ = rgbs[0].shape

# --- Step 4: Create CameraAZ objects ---
# intrinsics and extrinsics are per-frame (849 frames)
intrinsics = data['intrinsics']  # (849, 3, 3)
extrinsics = data['extrinsics']  # (849, 4, 4)
cameras = []
for i in range(intrinsics.shape[0]):
    intr = intrinsics[i]
    extr = extrinsics[i]
    # Normalize intrinsics by image size
    intr_normalized = {
        'fx': intr[0, 0] / imw,
        'fy': intr[1, 1] / imh,
        'cx': intr[0, 2] / imw,
        'cy': intr[1, 2] / imh,
        'k1': 0,
        'k2': 0,
    }
    # Create a CameraAZ instance (using the first 3 rows of extrinsics)
    cameras.append(CameraAZ(from_json={'extr': extr[:3], 'intr_normalized': intr_normalized}))

# --- Step 5: Create a Track3d object ---
# Here we pass the transposed 3D trajectories and corresponding visibility mask.
# The Track3d constructor will use:
#  - track3d.track3d: shape (num_points, num_frames, 3)  --> (11721, 849, 3)
#  - track3d.visible_list: shape (num_points, num_frames)   --> (11721, 849)
track3d = Track3d(
    tracks=None,          # Not needed since we pass track3d explicitly
    visibles=valids,      # 11721 x 849 visibility mask
    depths=depths,        # (849, imh, imw) depth maps for each frame
    cameras=cameras,      # List of 849 CameraAZ objects
    video=rgbs,           # (849, imh, imw, 3) RGB video frames
    track3d=trajs_3d,     # (11721, 849, 3) 3D point trajectories
    visible_list=valids,  # Use the same mask (or use 'visibs' if you prefer)
)

# --- Step 6: Call save_3d_track_vis ---
save_root = "data/pointodyssey/sample/r4_new_f/output"
vid = "r4_new_f"
prefix = "test"
os.makedirs(os.path.join(save_root, vid), exist_ok=True)

# This function will generate 3D track visualizations (and 2D if applicable)
save_3d_track_vis(track3d, rgbs, depths, save_root, vid, prefix)

plotting tracks: 100%|███████████████████████████████████████████| 849/849 [07:00<00:00,  2.02it/s]


: 

: 

: 

In [8]:
!pwd
!mv stereo4d-code stereo4d_code
!ls -lrt

/Users/kevin/Desktop/Kevin/projects/recon-2
mv: rename stereo4d-code to stereo4d_code/stereo4d-code: No such file or directory
total 1104
drwxr-xr-x  15 kevin  staff     480 Feb  2 23:59 utils
drwxr-xr-x   4 kevin  staff     128 Feb  3 00:52 tmp
drwxr-xr-x   2 kevin  staff      64 Feb  4 22:27 models
drwxr-xr-x   3 kevin  staff      96 Feb  5 00:07 notebooks
drwxr-xr-x  22 kevin  staff     704 Feb 13 21:32 datasets
drwxr-xr-x   5 kevin  staff     160 Mar  1 15:15 datasets_preprocess
drwxr-xr-x   9 kevin  staff     288 Mar  1 15:19 data
drwxr-xr-x  17 kevin  staff     544 Mar  3 14:37 stereo4d_code
-rw-r--r--   1 kevin  staff     876 Mar  3 14:45 pyproject.toml
-rw-r--r--   1 kevin  staff  559344 Mar  3 14:45 poetry.lock
